# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [27]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.0794147
 0.969709 
 0.3858   
 0.0784724
 0.430813 
 0.464975 
 0.740141 
 0.0131671
 0.293717 
 0.438812 
 0.501324 
 0.340205 
 0.156438 
 ⋮        
 0.532225 
 0.413748 
 0.683059 
 0.555187 
 0.727097 
 0.822645 
 0.71398  
 0.885196 
 0.205367 
 0.68485  
 0.354845 
 0.027818 

In [28]:
sum(a)   

5.002227731080686e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [29]:
#Pkg.add("BenchmarkTools")

In [30]:
using BenchmarkTools

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [31]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file
ClibUnix = replace(Clib, "C:", "/mnt/c")  # I've installed GCC on Linux subsystem
ClibUnix = replace(ClibUnix, '\\', '/')
# print(ClibUnix)  # for debug

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`bash -c """gcc -fPIC -O3 -msse3 -xc -shared -o $(ClibUnix * "." * Libdl.dlext) -"""`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [32]:
c_sum(a)

LoadError: [91merror compiling c_sum: could not load library "C:\Users\Owner\AppData\Local\Temp\jl_75BF.tmp"
%1 is not a valid Win32 application.
[39m

In [33]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

LoadError: [91merror compiling c_sum: could not load library "C:\Users\Owner\AppData\Local\Temp\jl_75BF.tmp"
%1 is not a valid Win32 application.
[39m

In [34]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [35]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true
```


We can now benchmark the C code directly from Julia:

In [36]:
c_bench = @benchmark c_sum($a) 

LoadError: [91merror compiling #_run#21: error compiling ##sample#665: error compiling ##core#664: could not load library "C:\Users\Owner\AppData\Local\Temp\jl_75BF.tmp"
%1 is not a valid Win32 application.
[39m

In [19]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

LoadError: [91mUndefVarError: c_bench not defined[39m

In [37]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

LoadError: [91mUndefVarError: c_bench not defined[39m

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [ ]:
#Pkg.add("PyCall")

In [7]:
using PyCall

In [9]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [10]:
pysum(a)

4.998345933891407e6

In [11]:
pysum(a) ≈ sum(a)

true

In [12]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     71.970 ms (0.00% GC)
  median time:      72.966 ms (0.00% GC)
  mean time:        73.451 ms (0.00% GC)
  maximum time:     83.488 ms (0.00% GC)
  --------------
  samples:          69
  evals/sample:     1

In [25]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 1 entry:
  "Python built-in" => 71.9697

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [21]:
using Conda 
#Conda.add("numpy")

In [22]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  22
  --------------
  minimum time:     12.724 ms (0.00% GC)
  median time:      12.963 ms (0.00% GC)
  mean time:        12.977 ms (0.00% GC)
  maximum time:     15.172 ms (0.00% GC)
  --------------
  samples:          385
  evals/sample:     1

In [23]:
numpy_sum(apy_list) # python thing

4.998345933891182e6

In [24]:
numpy_sum(apy_list) ≈ sum(a)

true

In [26]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "Python numpy"    => 12.7239
  "Python built-in" => 71.9697

# 4. Python, hand-written 

In [27]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x000000005F4B86D8>

In [28]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     599.171 ms (0.00% GC)
  median time:      601.071 ms (0.00% GC)
  mean time:        602.393 ms (0.00% GC)
  maximum time:     609.243 ms (0.00% GC)
  --------------
  samples:          9
  evals/sample:     1

In [29]:
sum_py(apy_list)

4.998345933891407e6

In [30]:
sum_py(apy_list) ≈ sum(a)

true

In [31]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "Python numpy"        => 12.7239
  "Python hand-written" => 599.171
  "Python built-in"     => 71.9697

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [32]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [33]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.754 ms (0.00% GC)
  median time:      5.908 ms (0.00% GC)
  mean time:        5.947 ms (0.00% GC)
  maximum time:     7.740 ms (0.00% GC)
  --------------
  samples:          839
  evals/sample:     1

In [34]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "Python numpy"        => 12.7239
  "Python hand-written" => 599.171
  "Python built-in"     => 71.9697
  "Julia built-in"      => 5.75383

# 6. Julia (hand-written) 

In [35]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [36]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.834 ms (0.00% GC)
  median time:      12.979 ms (0.00% GC)
  mean time:        13.056 ms (0.00% GC)
  maximum time:     14.530 ms (0.00% GC)
  --------------
  samples:          383
  evals/sample:     1

In [37]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "Python numpy"        => 12.7239
  "Julia hand-written"  => 12.8342
  "Python hand-written" => 599.171
  "Python built-in"     => 71.9697
  "Julia built-in"      => 5.75383

# Summary

In [38]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Julia built-in............5.75
Python numpy.............12.72
Julia hand-written.......12.83
Python built-in..........71.97
Python hand-written.....599.17
